<!-- label:sqlite -->
Opening an sqlite database

In [ ]:
import sqlite3
dbfilename = "/opt/data/finefoods.db"
dbcon = sqlite3.connect(dbfilename)

<!-- label:sqlite_firstquery -->
SQL query.

In [ ]:
cursor = dbcon.cursor()
sql = """
SELECT reviewhelpful, count(*)
FROM review
WHERE reviewscore < 2
GROUP BY reviewhelpful
"""
cursor.execute(sql)

<!-- label:sqlite_firstresults -->
Results can then be pulled from the database.

In [ ]:
from collections import Counter
ct = Counter()
for row_n, (score, count) in enumerate(cursor, 1):
    ct[score] = count
print(ct.most_common(n=3))

<!-- label:sqlite_secondquery -->
Some of the Python-side post-processing can be pushed
back to the database

In [ ]:
cursor = dbcon.cursor()
sql = """
SELECT reviewhelpful, count(*) AS ct
FROM review
WHERE reviewscore < 2
GROUP BY reviewhelpful
ORDER BY ct DESC
"""
cursor.execute(sql)
print(cursor.fetchmany(5))

<!-- label:sqlite_complexquery -->

In [ ]:
sql = """
select reviewhelpful, count(*)
from (select T.reviewername
      from (select reviewername, count(*) as reviewer_count
            from review
            group by reviewername) as T
      where T.reviewer_count > 5) as U
inner join review
on U.reviewername = review.reviewername
where reviewscore < 2
group by reviewhelpful
"""
cursor.execute(sql)


<!-- label:sqlalchemy_open -->
Opening the same database using an ORM (SQLalchemy).

In [ ]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

Base = automap_base()

# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine("sqlite:////opt/data/finefoods.db")

<!-- label:sqlalchemy_reflect -->
Use reflection on the SQL side to create the objects from the database.

In [ ]:
Base.prepare(engine, reflect=True)
review = Base.classes.review

---

<!-- label:sqlalchemy_query -->
Make a query using SQLalchemy's methods.

In [ ]:
session = Session(engine)
from sqlalchemy import func # SQL functions

query = (session
         .query(review.reviewhelpful,
                func.count(review.reviewhelpful))
         .filter(review.reviewscore < 2)
         .group_by(review.reviewhelpful)
         .order_by(func.count(review.reviewhelpful).desc()))

res = query.all()
res[:3]

How were we doing before StackOverflow ?

---

<!-- label:sqlalchemy_sql -->
Function composition is generating SQL code.

In [ ]:
from sqlalchemy.dialects import sqlite
print(str(query.statement.compile(dialect=sqlite.dialect())))

---

<!-- label:dplyr_rpy2_string -->
<!-- config:two-columns -->

With dplyr, an SQL table is a data table.

In [ ]:
from rpy2.robjects import r

r_code = """
suppressMessages(require("dplyr"))
dbfilename <- '""" + dbfilename + """'
datasrc <- src_sqlite(dbfilename)
review_tbl <- tbl(datasrc, "review")

res <- filter(review_tbl,
              reviewscore < 2) %>%
       count(reviewhelpful) %>%
       arrange('desc(n)')
"""

res = r(r_code)
print(res)

---

<table>
<tr>
  <td>We traded the knowledge of SQL for </td>
  <td><strike>the</strike></td>
  <td> knowledge of R.</td>
</tr>
<tr>
  <td></td>
  <td>a little</td>
  <td></td>
</tr>
</table>

---

<!-- label:dplyr_table -->
dplyr is not trying to map objects. It is focusing on databases
as sources of tables.

In [ ]:
from rpy2.robjects.lib import dplyr


datasrc  = dplyr.src_sqlite(dbfilename)
review_tbl = datasrc.get_table("review")

<!-- label:dplyr_query -->
The table can be queried using the dplyr interface.

In [ ]:
res =  (review_tbl
        .filter('reviewscore < 2')
        .count('reviewhelpful')
        .arrange('desc(n)'))

print(res)


Strings are snippets of R code for dplyr.

R can be considered a domain-specific language (DSL) in the Python code.

---

<!-- label:ggplot2_figure -->
The R package ggplot2 can also be used.

In [ ]:
from rpy2.robjects import r, globalenv
import rpy2.robjects.lib.ggplot2 as gg

split_review = \
    'as.integer(sapply(strsplit(reviewhelpful, "/"), "[", %i))'

p = (gg.ggplot((res.collect().
                mutate(r1 = split_review % 1,
                       r2 = split_review % 2))) +
     gg.aes_string(x='r1+1', y='r2+1', size='n') +
     gg.geom_point(alpha = 0.5) +
     gg.scale_x_log10("review voted helpful") +
     gg.scale_y_log10("review") +
     gg.scale_size(trans="sqrt"))

<!-- label:ggplot2_plot -->
<!-- config:split-output -->
Sending the resulting figure to a jupyter notebook output.

In [ ]:
from rpy2.robjects.lib import grdevices

with grdevices.render_to_bytesio(grdevices.png, 
                                 width=800,
                                 height=600, 
                                 res=120) as b:
    p.plot()

from IPython.display import Image, display
data = b.getvalue()
display(Image(data=data, format='png', embed=True))

---

<!-- label:bokeh -->
<!-- config:split-output -->

In [ ]:
from bokeh.plotting import figure
from bokeh.plotting import figure, show, output_notebook
output_notebook()

plot = figure()
res =  (review_tbl
        .filter('reviewscore < 2')
        .count('reviewhelpful')
        .collect())

import math
plot.scatter(list(int(x.split('/')[0]) \
                  for x in res.rx2('reviewhelpful')),
             list(int(x.split('/')[1]) \
                  for x in res.rx2('reviewhelpful')),
             radius=list(math.log10(x/100) for x in res.rx2('n')),
             fill_alpha=.3)

In [ ]:
show(plot)


---

<!-- label:spark_setup -->
Spark can be started from regular Python code.

In [ ]:
import findspark
findspark.init()

import pyspark

conf = pyspark.conf.SparkConf()
(conf.setMaster('local[2]')
 .setAppName('ipython-notebook')
 .set("spark.executor.memory", "2g"))

sc = pyspark.SparkContext(conf=conf)

<!-- label:spark_dataframe -->

In [ ]:
from pyspark.sql import SQLContext, Row
sqlcontext = SQLContext(sc)
cursor.execute('select * from review limit 10000')
review = \
    sqlcontext.createDataFrame(cursor,
                               tuple(x[0] for x in cursor.description))
review.registerTempTable("review")

---

<!-- label:spark_query -->
SQL can be used to query the data.

In [ ]:
sql = """
SELECT reviewhelpful, count(*) AS ct
FROM review
WHERE reviewscore < 2
GROUP BY reviewhelpful
ORDER BY ct DESC
"""
counts = sqlcontext.sql(sql)

<!-- label:spark_query_collect -->
The evaluation is only performed when the results are needed.

In [ ]:
res = counts.collect()
res[:3]

<!-- label:spark_mapreduce -->
Map/reduce is at the heart of Spark.

In [ ]:
names = (review
         .flatMap(lambda p: p.reviewsummary.split())
         .map(lambda word: (word.lower(), 1))
         .reduceByKey(lambda a, b: a+b))
names.takeOrdered(15, key = lambda x: -x[1])

---

<!-- label:spark_sqlmapreduce -->

In [ ]:
sql = """
SELECT *
FROM review
WHERE reviewscore < 2
"""
lowscore = sqlcontext.sql(sql)
names = (lowscore
         .flatMap(lambda p: p.reviewsummary.split())
         .map(lambda word: (word.lower(), 1))
         .reduceByKey(lambda a, b: a+b))
names.takeOrdered(8, key = lambda x: -x[1])

---

<!-- label:spark_sqlmapreduceggplot -->

In [ ]:
lst = names.takeOrdered(8, key = lambda x: -x[1])
from rpy2.robjects.vectors import StrVector, IntVector
dataf = dplyr.DataFrame({'word': StrVector([x[0] for x in lst]),
                         'count': IntVector([x[1] for x in lst])})
p = (gg.ggplot(dataf) +
     gg.geom_bar(gg.aes_string(x='word', y='count'),
                 stat="identity"))

with grdevices.render_to_bytesio(grdevices.png, 
                                 width=800,
                                 height=600, 
                                 res=120) as b:
    p.plot()
display(Image(data=b.getvalue(), format='png', embed=True))
